In [15]:
from utils import random_idx
from utils import utils
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import sys
import Queue

k = 5000
N = 10000
cluster_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
ordered = 1
alph = string.lowercase + ' '

def create_lang_vec(cluster_sizes, N=N, k=k):
    total_lang = np.zeros((1,N))
    # generate english vector
    for cz in cluster_sizes:
        lang_vector = random_idx.generate_RI_text_fast(N, RI_letters, cz, ordered, "preprocessed_texts/english/alice_in_wonderland.txt", alph)
        total_lang += lang_vector
    return total_lang

def plot_letter_frequencies(frequencies, title):
    x = np.linspace(0, 26, num=26)
    labels = [letter for letter in alph]
    plt.figure()
    plt.bar(x, frequencies)
    plt.xticks(x, labels, rotation='horizontal')
    plt.xlabel('Letters')
    plt.ylabel('Frequencies')
    plt.title(title)
    plt.show()
"""
Clipping basically means limiting or saturating the max/min value of each element in a vector. 
In other words, we want to reduce the precision of vector. 
You can use a function like below to limit the max/min value of each element: n is the input value, 
and bitWidth is the number of bits that can be used to represent each element 
(bitWidth of 1 makes the vector as a binary vector). 

"""
def clamp(lang_vec, Min, Max):
    for i in range(0, len(lang_vec)):
        for j in range(0, len(lang_vec[0])):
            if lang_vec[i][j] > Max:
                lang_vec[i][j] = Max
            elif lang_vec[i][j] < Min:
                lang_vec[i][j] = Min
    return lang_vec

def clamp_to_binary(lang_vec):
    #print lang_vec
    for i in range(0, len(lang_vec)):
        for j in range(0, len(lang_vec[0])):
            if lang_vec[i][j] >= 0:
                lang_vec[i][j] = 1
            else:
                lang_vec[i][j] = -1
    return lang_vec

# predicts the most likely letter for each prefix in the word
def predict(word, clamped, type_clamped):
    for i in range(len(word)):
        prefix = random_idx.id_vector(N, word[:i], alph, RI_letters, ordered)
        sprefix = np.roll(prefix, 1)
        prefix_ngram = np.multiply(clamped[len(word[:i])+1], sprefix)
        frequencies = []
        for j in range(26):
            frequencies.append(np.dot(prefix_ngram, RI_letters[j]))
        plot_letter_frequencies(frequencies, type_clamped + " for predicting the next letter prefix " + word[:i] + " in " + word)  

RI_letters = random_idx.generate_letter_id_vectors(N, k, alph)
# lang_vectors in sizes 1-8
lang_vectors = []
for size in cluster_sizes:
    lang_vectors.append(create_lang_vec([size]))
lang_vectors.insert(0, np.zeros((1,N)))

search_words = ["foot", "runs", "consider", "vanish", "the", "she", "lady"]
clusters = [np.zeros((1,N)), lang_vectors[1]]
for i in range(2, len(lang_vectors)):
    clusters.append(np.add(clusters[i-1], lang_vectors[i]))
    
clamped_to_binary_lang_vecs = [clamp_to_binary(lv) for lv in lang_vectors]
# chosen from the number of letters in Alice. 
# An n-gram can't show up more times than the total number of letters in the text.
clamped_lang_vecs = [clamp(lv, -106036, 106036) for lv in lang_vectors]
"""
for word in search_words:
    predict(word, clamped_to_binary_lang_vecs, "binary")
    predict(word, clamped_lang_vecs, "arbitrary cutoff")
"""


'\nfor word in search_words:\n    predict(word, clamped_to_binary_lang_vecs, "binary")\n    predict(word, clamped_lang_vecs, "arbitrary cutoff")\n'

In [ ]:
# Creates the load log vector for specified ngrams
def load_log_vector(filepath, cluster_sizes, N=N, k=k):
    f = open(filepath, "r")
    text = f.read()
    f.close()
    lang_vecs = []
    ngrams_freq = {}
    for cz in cluster_sizes:
        total_lang = np.zeros((1,N))
        for i in range(len(text)):
            if i + cz <= len(text):
                word = text[i:i+cz]
                if word in ngrams_freq.keys():
                    ngrams_freq[word] += 1
                else:
                    ngrams_freq[word] = 1
                    
                word_vec = random_idx.id_vector(N, word, alph, RI_letters, ordered)
                total_lang += np.log2(ngrams_freq[word])*word_vec
        lang_vecs.append(total_lang)
    return lang_vecs

# predicts the most likely letter for each prefix in the word
def predict2(word, lang_vecs):
    for i in range(len(word)):
        prefix = random_idx.id_vector(N, word[:i], alph, RI_letters, ordered)
        sprefix = np.roll(prefix, 1)
        prefix_ngram = np.multiply(lang_vecs[len(word[:i])+1], sprefix)
        frequencies = []
        for j in range(26):
            frequencies.append(np.dot(prefix_ngram, RI_letters[j]))
        plot_letter_frequencies(frequencies, "log language vectors predict the next letter for prefix " + word[:i] + " in " + word)  

# lang_vectors in sizes 1-8
log_lang_vecs = load_log_vector("preprocessed_texts/english/alice_in_wonderland.txt", cluster_sizes)
predict("consider", log_lang_vecs)